# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from urllib.parse import urljoin

def scrape_books(min_rating, max_price):
    """
    Scrape books from Books to Scrape website with specified minimum rating and maximum price.
    
    Args:
        min_rating (float): Minimum rating (1-5 stars)
        max_price (float): Maximum price in pounds
    
    Returns:
        pandas.DataFrame: DataFrame with columns UPC, Title, Price (£), Rating, Genre, Availability, Description
    """
    
    base_url = "https://books.toscrape.com/"
    books_data = []
    page = 1
    
    print(f"Starting scrape for books with rating >= {min_rating} and price <= £{max_price}")
    
    while True:
        # Construct URL for current page
        if page == 1:
            url = base_url
        else:
            url = f"{base_url}catalogue/page-{page}.html"
        
        print(f"Scraping page {page}...")
        
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error accessing page {page}: {e}")
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all book articles on the page
        books = soup.find_all('article', class_='product_pod')
        
        if not books:
            print("No more books found. Ending scrape.")
            break
        
        for book in books:
            try:
                # Extract basic info from listing page
                title = book.find('h3').find('a')['title']
                
                # Extract price
                price_text = book.find('p', class_='price_color').text
                price = float(re.search(r'£(\d+\.\d+)', price_text).group(1))
                
                # Extract rating
                rating_element = book.find('p', class_='star-rating')
                rating_class = rating_element.get('class')[1] if rating_element else None
                
                rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
                rating = rating_map.get(rating_class, 0)
                
                # Apply filters
                if rating >= min_rating and price <= max_price:
                    # Get detailed book page URL
                    book_link = book.find('h3').find('a')['href']
                    book_url = urljoin(base_url + "catalogue/", book_link)
                    
                    # Scrape detailed information
                    book_details = scrape_book_details(book_url)
                    
                    if book_details:
                        book_data = {
                            'UPC': book_details.get('UPC', 'N/A'),
                            'Title': title,
                            'Price (£)': price,
                            'Rating': rating,
                            'Genre': book_details.get('Genre', 'N/A'),
                            'Availability': book_details.get('Availability', 'N/A'),
                            'Description': book_details.get('Description', 'N/A')
                        }
                        books_data.append(book_data)
                        print(f"Added: {title} (Rating: {rating}, Price: £{price})")
                
            except Exception as e:
                print(f"Error processing book: {e}")
                continue
        
        page += 1
        # Add small delay to be respectful to the server
        time.sleep(0.5)
        
        # Safety check to avoid infinite loop
        if page > 50:  # Books to Scrape has ~50 pages
            break
    
    print(f"\nScraping complete! Found {len(books_data)} books matching criteria.")
    
    if books_data:
        df = pd.DataFrame(books_data)
        return df
    else:
        return pd.DataFrame(columns=['UPC', 'Title', 'Price (£)', 'Rating', 'Genre', 'Availability', 'Description'])

def scrape_book_details(book_url):
    """
    Scrape detailed information from a book's individual page.
    
    Args:
        book_url (str): URL of the book's detail page
    
    Returns:
        dict: Dictionary containing detailed book information
    """
    try:
        response = requests.get(book_url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        details = {}
        
        # Extract UPC from product information table
        product_info = soup.find('table', class_='table-striped')
        if product_info:
            rows = product_info.find_all('tr')
            for row in rows:
                th = row.find('th')
                td = row.find('td')
                if th and td:
                    if 'UPC' in th.text:
                        details['UPC'] = td.text.strip()
                    elif 'Availability' in th.text:
                        details['Availability'] = td.text.strip()
        
        # Extract genre from breadcrumb
        breadcrumb = soup.find('ul', class_='breadcrumb')
        if breadcrumb:
            genre_links = breadcrumb.find_all('a')
            if len(genre_links) >= 3:  # Home > Books > Genre > Book
                details['Genre'] = genre_links[2].text.strip()
        
        # Extract description
        description_div = soup.find('div', id='product_description')
        if description_div:
            description_p = description_div.find_next_sibling('p')
            if description_p:
                details['Description'] = description_p.text.strip()
        
        return details
        
    except Exception as e:
        print(f"Error scraping book details from {book_url}: {e}")
        return {}

# Test the function with the specified parameters
print("Testing scrape_books function...")
df = scrape_books(min_rating=4.0, max_price=20.0)

# Display results
print(f"\nResults Summary:")
print(f"Total books found: {len(df)}")

if not df.empty:
    print(f"\nFirst 5 books:")
    print(df.head())
    
    print(f"\nDataFrame Info:")
    print(df.info())
    
    print(f"\nRating distribution:")
    print(df['Rating'].value_counts().sort_index())
    
    print(f"\nPrice statistics:")
    print(df['Price (£)'].describe())
    
    print(f"\nGenre distribution:")
    print(df['Genre'].value_counts().head(10))
else:
    print("No books found matching the criteria.")

# Save to CSV for further analysis
if not df.empty:
    df.to_csv('books_scraped_data.csv', index=False)
    print(f"\nData saved to 'books_scraped_data.csv'")

Testing scrape_books function...
Starting scrape for books with rating >= 4.0 and price <= £20.0
Scraping page 1...
Error scraping book details from https://books.toscrape.com/catalogue/catalogue/set-me-free_988/index.html: 404 Client Error: Not Found for url: https://books.toscrape.com/catalogue/catalogue/set-me-free_988/index.html
Error scraping book details from https://books.toscrape.com/catalogue/catalogue/set-me-free_988/index.html: 404 Client Error: Not Found for url: https://books.toscrape.com/catalogue/catalogue/set-me-free_988/index.html
Scraping page 2...
Scraping page 2...
Added: The Four Agreements: A Practical Guide to Personal Freedom (Rating: 5, Price: £17.66)
Added: The Four Agreements: A Practical Guide to Personal Freedom (Rating: 5, Price: £17.66)
Added: Sophie's World (Rating: 5, Price: £15.94)
Added: Sophie's World (Rating: 5, Price: £15.94)
Scraping page 3...
Scraping page 3...
Added: Untitled Collection: Sabbath Poems 2014 (Rating: 4, Price: £14.27)
Added: Untit